# AutoGen Basic Sample

In this example, you will use the [AutoGen](https://aka.ms/ai-agents/autogen) AI Framework to build a simple agent.

The purpose of this example is to demonstrate the steps that will later be applied in other examples when implementing various agent patterns.


## Import the Needed Python Packages


In [ ]:
import os
from dotenv import load_dotenv

from autogen_agentchat.agents import AssistantAgent
from autogen_core.models import UserMessage
from autogen_ext.models.azure import AzureAIChatCompletionClient
from azure.core.credentials import AzureKeyCredential
from autogen_core import CancellationToken

from autogen_agentchat.messages import TextMessage
from autogen_agentchat.ui import Console


## Create the Client

In this example, we will use [GitHub Models](https://aka.ms/ai-agents-beginners/github-models) to access the LLM.

The `model` is specified as `gpt-4o-mini`. You can experiment by switching to another model available in the GitHub Models marketplace to observe varying outcomes.

For a quick test, we'll use a simple prompt: `What is the capital of France`.


In [ ]:
load_dotenv()
client = AzureAIChatCompletionClient(
    model="gpt-4o-mini",
    endpoint="https://models.inference.ai.azure.com",
    # To authenticate with the model you will need to generate a personal access token (PAT) in your GitHub settings.
    # Create your PAT token by following instructions here: https://docs.github.com/en/authentication/keeping-your-account-and-data-secure/managing-your-personal-access-tokens
    credential=AzureKeyCredential(os.getenv("GITHUB_TOKEN")),
    model_info={
        "json_output": True,
        "function_calling": True,
        "vision": True,
        "family": "unknown",
    },
)

result = await client.create([UserMessage(content="What is the capital of France?", source="user")])
print(result)

## Defining the Agent

Now that the `client` is set up and confirmed to be working, let's create an `AssistantAgent`. Each agent can be assigned the following:  
**name** - A shorthand name that will be helpful for referencing it in multi-agent workflows.  
**model_client** - The client you created in the previous step.  
**tools** - Tools available for the Agent to use in completing a task.  
**system_message** - The metaprompt that defines the task, behavior, and tone of the LLM.  

You can modify the system message to observe how the LLM responds. We will discuss `tools` in Lesson #4.


In [ ]:
agent = AssistantAgent(
    name="assistant",
    model_client=client,
    tools=[],
    system_message="You are a travel agent that plans great vacations",
)

## Run the Agent

The function below will execute the agent. We use the `on_message` method to update the Agent's state with the new message.

In this example, we update the state with a new message from the user, which is `"Plan me a great sunny vacation"`.

You can modify the message content to observe how the LLM responds differently.


In [ ]:
from IPython.display import display, HTML


async def assistant_run():
    # Define the query
    user_query = "Plan me a great sunny vacation"

    # Start building HTML output
    html_output = "<div style='margin-bottom:10px'>"
    html_output += "<div style='font-weight:bold'>User:</div>"
    html_output += f"<div style='margin-left:20px'>{user_query}</div>"
    html_output += "</div>"

    # Execute the agent response
    response = await agent.on_messages(
        [TextMessage(content=user_query, source="user")],
        cancellation_token=CancellationToken(),
    )

    # Add agent response to HTML
    html_output += "<div style='margin-bottom:20px'>"
    html_output += "<div style='font-weight:bold'>Assistant:</div>"
    html_output += f"<div style='margin-left:20px; white-space:pre-wrap'>{response.chat_message.content}</div>"
    html_output += "</div>"

    # Display formatted HTML
    display(HTML(html_output))

# Run the function
await assistant_run()


---

**Disclaimer**:  
This document has been translated using the AI translation service [Co-op Translator](https://github.com/Azure/co-op-translator). While we aim for accuracy, please note that automated translations may include errors or inaccuracies. The original document in its native language should be regarded as the authoritative source. For critical information, professional human translation is advised. We are not responsible for any misunderstandings or misinterpretations resulting from the use of this translation.
